# Acript to make taptest measurements into channel position metadata.

In [1]:
%matplotlib inline
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import detrend
from obspy.signal.filter import bandpass as bp
from obspy import UTCDateTime
import dascore as dc
import pyproj

## get file information

In [ ]:
def get_tstamp(fname):
    datestr = fname.split('_')[1].split('-')
    y = int(datestr[0])
    m = int(datestr[1])
    d = int(datestr[2])
    timestr = fname.split('_')[2].split('.')
    H = int(timestr[0])
    M = int(timestr[1])
    S = int(timestr[2])
    return UTCDateTime('%04d-%02d-%02dT%02d:%02d:%02d' % (y,m,d,H,M,S))

fdir = '/Volumes/PeatDAS/harper_test_data'
flist = np.array(os.listdir(fdir))
ftime = np.array([get_tstamp(fname) for fname in flist])
index = np.argsort(np.array(ftime)-ftime[0])
flist = flist[index]
ftime = ftime[index]

## Get acquisition parameters

In [ ]:
fname = flist[0]
with h5py.File(os.path.join(fdir,fname),'r') as fp:
    GL = fp['Acquisition'].attrs['GaugeLength']
    dx = fp['Acquisition'].attrs['SpatialSamplingInterval']
    fs = fp['Acquisition']['Raw[0]'].attrs['OutputDataRate']
    nx = fp['Acquisition']['Raw[0]'].attrs['NumberOfLoci']
    ns = len(fp['Acquisition']['Raw[0]']['RawDataTime'][:])
    #data = fp['Acquisition']['Raw[0]']['RawData'][:]
    print(fname)
    print('Gauge length (m):',GL)
    print('Channel spacing (m):',dx)
    print('Sampling rate (Hz):',fs)
    print('Num channels:',nx)
    print('Num samples:',ns)

# Convert corners and tap-test files into usable metadata

This script will create a metadata CSV file with the following attributes:
    chanID, longitude, latitude, elevation, easting, northing

In [4]:
df = pd.read_csv("corners-joesfield_final_gps.csv")
df.head()

,time,name,longitude,latitude,elevation
0,2023-03-16T15:58:01,0,-2.605449,52.916499,104.155931
1,2023-03-16T15:58:12,1,-2.605495,52.916499,104.149541
2,2023-03-16T16:01:32,2,-2.608302,52.915618,97.887184
3,2023-03-16T16:01:49,3,-2.608213,52.915503,97.993868
4,2023-03-16T16:02:46,4,-2.607865,52.915749,99.280637


first convert the lat long into NE

In [19]:
from pyproj import Proj
ZoneNo = "29" #Manually input, find the zone here: https://www.dmap.co.uk/utmworld.htm
myProj = Proj("+proj=utm +zone="+ZoneNo+" +north,\
 +ellps=WGS84 +datum=WGS84 +units=m +no_defs") #north for north hemisphere
UTMx, UTMy = myProj(df["longitude"], df["latitude"])
print(UTMx)
df["easting"]=UTMx
df["northing"]=UTMy
df.head()


[929723.67041316 929720.584497   929540.91893111 929548.07302359
 929568.94206158 929573.98669892 929578.78457273 929620.02162467
 929609.59900266 929621.12865417 929656.4149598  929645.87966985
 929658.06742822 929697.052059   929687.42993799 929698.36127268
 929732.00628519 929728.21799463]


,time,name,longitude,latitude,elevation,easting,northing
0,2023-03-16T15:58:01,0,-2.605449,52.916499,104.155931,929723.670413,5.882146e+06
1,2023-03-16T15:58:12,1,-2.605495,52.916499,104.149541,929720.584497,5.882146e+06
2,2023-03-16T16:01:32,2,-2.608302,52.915618,97.887184,929540.918931,5.882031e+06
3,2023-03-16T16:01:49,3,-2.608213,52.915503,97.993868,929548.073024,5.882019e+06
4,2023-03-16T16:02:46,4,-2.607865,52.915749,99.280637,929568.942062,5.882049e+06
